In [2]:
import requests
from time import sleep
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

In [5]:
def main_request(url, element, user_agent):
    params = {'startDate': '2019-01-01', 'endDate': '2020-12-08', 'matchType': 'Online', 'offset': str(element), 'content': 'stats'}
    return requests.get(url + '/results', params = params, headers = {'User-Agent': user_agent})

def mini_request(url, user_agent):
    return requests.get(url, headers = {'User-Agent': user_agent})

In [6]:
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'
url = 'https://www.hltv.org'

r = main_request(url, 0, user_agent)
soup = BeautifulSoup(r.content, 'html.parser')

numb_elem = int(soup.select_one('.pagination-data').text.split()[-1])
interval = int(soup.select_one('.pagination-data').text.split()[2])

main_list = []
map_list = []
stats_list = []

for elem in range(0, numb_elem, interval):
    page = main_request(url, elem, user_agent)
    if page.status_code == 200:
        soup_page = BeautifulSoup(page.content, 'html.parser')

        body = soup_page.select('.results-holder')
        for i in range(len(body)):
            if len(body[i].select('.results-all')) > 0:
                matches = body[i].select('.results-all')[0].select('a')
        
        for i, match in enumerate(matches):
            match_href = match['href']
            match_page = mini_request(url + match_href, user_agent)
            if match_page.status_code == 200:
                soup_match_page = BeautifulSoup(match_page.content, 'html.parser')
                body = soup_match_page.select_one('.match-page')
                date = body.select_one('.date').text.strip()
                time = body.select_one('.time').text.strip()
                team1 = body.select_one('.team1-gradient a').text.strip()
                result1 = body.select_one('.team1-gradient').text.strip()[-1]
                team2 = body.select_one('.team2-gradient a').text.strip()
                result2 = body.select_one('.team2-gradient').text.strip()[-1]
                event = body.select_one('.event a').text.strip()
                event_href = body.select_one('.event a')['href']
                main_list.append([match_href, date, time, team1, result1, team2, result2, event, event_href])
                
                event_page = mini_request(url + event_href, user_agent)
                if event_page.status_code == 200:
                    soup_event_page = BeautifulSoup(event_page.content, 'html.parser')
                    prizepool = soup_event_page.select_one('tbody').select_one('.prizepool').text.strip()
                else:
                    prizepool = 'error'
                    print('Ошибка ' + str(page.status_code) + ' на странице с элемента ' + str(elem) + ' матча ' + match_href + 'для турнира')
                main_list[elem + i].append(prizepool)
                                
                maps_body = body.select_one('.col-6')
                if maps_body.select_one('.padding') != None:
                    star = maps_body.select_one('.padding').text.strip()
                else:
                    star = 'error'
                for map_n in maps_body.select('.mapholder'):
                    map_name = map_n.select_one('.mapname').text.strip()
                    if map_n.select_one('.results-left') != None:
                        result1 = map_n.select_one('.results-left').select_one('.results-team-score').text.strip()
                        result2 = map_n.select_one('.results-right').select_one('.results-team-score').text.strip()
                        if len(map_n.select('.pick')) > 0:
                            pick = map_n.select_one('.pick').select_one('.results-teamname').text.strip()
                        else:
                            pick = 'dec'
                        if map_n.select_one('.results-center-half-score') != None:
                            half_results = map_n.select_one('.results-center-half-score').text.strip()
                        else:
                            half_results = '-'
                    else:
                        result1 = 0
                        result2 = 0
                        pick = 0
                        half_results = 0
                        star = 0
                    map_list.append([match_href, map_name, result1, result2, pick, half_results, star])
                
                if body.select_one('.matchstats') != None:
                    match_stats = body.select_one('.matchstats').select('.totalstats')
                    for table in match_stats:
                        team_name = table.select_one('.align-logo').text.strip()
                        rows = table.select('tr')[1:]
                        for row in rows:
                            player_name = row.select_one('span').text.strip()
                            player_kd = row.select_one('.kd').text.strip()
                            if row.select_one('.adr') != None:
                                player_adr = row.select_one('.adr').text.strip()
                            else:
                                player_adr = 'error'
                            player_rating = row.select_one('.rating').text.strip()
                            stats_list.append([match_href, team_name, player_name, player_kd, player_adr, player_rating]) 
                else:
                    team_name = 0
                    player_name = 0
                    player_kd = 0
                    player_adr = 0
                    player_rating = 0
                    stats_list.append([match_href, team_name, player_name, player_kd, player_adr, player_rating])
            else:
                print('Ошибка ' + str(page.status_code) + ' на странице с элемента ' + str(elem) + ' матча ' + match_href)
    else:
        print('Ошибка ' + str(page.status_code) + ' на странице с элемента ' + str(elem))
    
    df_main = pd.DataFrame(main_list, columns = ['match_href', 'date', 'time', 'team1', 'result1', 'team2', 'result2', 'event', 'event_href', 'prizepool'])
    df_maps = pd.DataFrame(map_list, columns = ['match_href', 'map_name', 'result1', 'result2', 'pick', 'half_results', 'star'])
    df_stats = pd.DataFrame(stats_list, columns = ['match_href', 'team_name', 'player_name', 'player_kd', 'player_adr', 'player_rating'])

    df_maps['star'] = df_maps['star'].apply(lambda x: str(x))
    df_maps['half_results'] = df_maps['half_results'].apply(lambda x: str(x))
    df_maps['star'] = df_maps['star'].apply(lambda x: x.replace('\n', ''))
    df_maps['half_results'] = df_maps['half_results'].apply(lambda x: x.replace(';', ','))
    
    df_main.to_csv('C://data/hltv/main.csv', index=False)
    df_maps.to_csv('C://data/hltv/maps.csv', index=False)
    df_stats.to_csv('C://data/hltv/stats.csv', index=False)
    print(elem)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
